In [1]:
from __future__ import print_function, division
from torchvision import datasets, models, transforms
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.autograd import Variable
import numpy as np
import torchvision
from torchvision import models, transforms
import torchvision.datasets as dsets
import matplotlib.pyplot as plt
import time
import os
import copy

plt.ion()  

use_gpu = torch.cuda.is_available()
if use_gpu:
    print("Using CUDA")

Using CUDA


# Loading the data which has folder name as a label and do transformation

In [2]:
train_dir = './fruits-360/Training/'
normalize = transforms.Normalize(mean=[0.0, 0.0, 0.0],
                                 std=[1, 1, 1])

train_dataset = dsets.ImageFolder(
    train_dir,
    transforms.Compose([
        # Data augmentation is a good practice for the train set
        # Here, we randomly crop the image to 224x224 and
        # randomly flip it horizontally. 
     #transforms.ToPILImage(),
     transforms.Resize((224,224)),
     transforms.ToTensor(),
     transforms.Normalize((0.5,0.5,0.5), (0.5, 0.5, 0.5)),
    ]))
print(len(train_dataset))

53177


# HyperParam Setting

In [3]:
batch_size = 8
validation_split = .2
shuffle_dataset = True
random_seed= 42

# Diving data into train and validation

In [4]:
from torch.utils.data.sampler import SubsetRandomSampler

# Creating data indices for training and validation splits:
dataset_size = len(train_dataset)
indices = list(range(dataset_size))
split = int(np.floor(validation_split * dataset_size))
if shuffle_dataset :
    np.random.seed(random_seed)
    np.random.shuffle(indices)
train_indices, val_indices = indices[split:], indices[:split]

# Creating PT data samplers and loaders:
train_sampler = SubsetRandomSampler(train_indices)
valid_sampler = SubsetRandomSampler(val_indices)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, 
                                           sampler=train_sampler)
validation_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size,
                                                sampler=valid_sampler)

# Lenght of loaders

In [5]:
#print("num of classes ",train_loader.classes)
print(len(train_dataset))
print(len(train_loader))
print(len(validation_loader))

53177
5318
1330


# freezing feature extractor layer.

In [6]:
def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False

# Loading the model

In [7]:
model = models.resnet18(pretrained=False)
#set_parameter_requires_grad(model, True)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 130)
input_size = 224

In [8]:
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
criteria=nn.CrossEntropyLoss()
model = model.cuda()

# Training the model

In [9]:
#import skimage
for epoch in range(5):
    ave_loss=0.0
    
    for i,data in enumerate(train_loader,0):
        inputs,labels =data
        #print(inputs.shape)
        inputs=torch.from_numpy(np.array(inputs))
        labels=torch.from_numpy(np.array(labels))
        inputs = inputs.cuda()
        labels = labels.cuda()
        optimizer.zero_grad()
        output=model(inputs)
        
        
        loss=criteria(output,labels)
        loss.backward()
        optimizer.step()
        ave_loss+=loss.item()
        if i%500==499:    # print every 200 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, ave_loss / 100))
            ave_loss = 0.0
correct=0
total=0


[1,   500] loss: 17.531
[1,  1000] loss: 9.812
[1,  1500] loss: 6.963
[1,  2000] loss: 4.993
[1,  2500] loss: 3.947
[1,  3000] loss: 3.033
[1,  3500] loss: 2.210
[1,  4000] loss: 1.788
[1,  4500] loss: 1.389
[1,  5000] loss: 1.273
[2,   500] loss: 0.901
[2,  1000] loss: 0.686
[2,  1500] loss: 0.685
[2,  2000] loss: 0.564
[2,  2500] loss: 0.474
[2,  3000] loss: 0.589
[2,  3500] loss: 0.326
[2,  4000] loss: 0.308
[2,  4500] loss: 0.307
[2,  5000] loss: 0.326
[3,   500] loss: 0.226
[3,  1000] loss: 0.135
[3,  1500] loss: 0.108
[3,  2000] loss: 0.152
[3,  2500] loss: 0.177
[3,  3000] loss: 0.120
[3,  3500] loss: 0.163
[3,  4000] loss: 0.135
[3,  4500] loss: 0.200
[3,  5000] loss: 0.162
[4,   500] loss: 0.050
[4,  1000] loss: 0.141
[4,  1500] loss: 0.050
[4,  2000] loss: 0.021
[4,  2500] loss: 0.064
[4,  3000] loss: 0.059
[4,  3500] loss: 0.068
[4,  4000] loss: 0.049
[4,  4500] loss: 0.082
[4,  5000] loss: 0.091
[5,   500] loss: 0.077
[5,  1000] loss: 0.016
[5,  1500] loss: 0.041
[5,  2000]

# Testing the model

In [10]:
with torch.no_grad():
    for inputs,labels in validation_loader:
#         inputs=data['image']
#         labels=data['label']
        
        inputs=torch.from_numpy(np.array(inputs))
        labels=torch.from_numpy(np.array(labels))
        outputs=model(inputs.cuda())
        _,predicted=torch.max(outputs.data,1)
        total+=labels.size(0)
        correct+=(predicted==labels.cuda()).sum().item()

print('Accuracy of the network on the  test images: %0.6f %%' % (
    100 * correct / total))


Accuracy of the network on the  test images: 99.943583 %
